### ` REGRESSION LOGISTIQUE `

In [1]:
import numpy as np

# fonction logistique
def logistic(x):
    return 1/(1+np.exp(-x))
    
# logarithme de la vraissemlance (fonction d'evaluation)
def log_vraissemblance(X, y, b):
    a = logistic(np.dot(X, b))
    return np.dot(y, np.log(a)) + np.dot(1 - y, np.log(1 - a))
    
# gradient du logarithme de la vraissemblance 
def grad_log_vraissemblance(X, y, b):
    a = logistic(np.dot(X, b))
    return np.dot(X.T, y - a)

# fonction d'evaluation du taux d'erreur avec la fonction logistique
def rss(X, y, b):
    a = y - logistic(np.dot(X, b))
    return np.dot(a.T, a) / a.shape[0]
    
# gradient de rss
def grad_rss(X, y, b):
    a = logistic(np.dot(X, b))
    return  2 * np.dot(X.T, (y - a)*(a - 1))

#algorithme de descente pour les problemes de minimisation et maximisation
def gradient_descent(gradient, X, y, start,learning_rate, n_iter, tolerance, minProblem = True):
    vector = start
    
    # si c'est un probleme de maximisation on prend le gradient comme direction de
    coef = 1
    if minProblem :
        coef = -1
        
    for _ in range(n_iter):
        diff = coef*learning_rate*gradient(X,y,vector)
        if np.all(np.abs(diff)<=tolerance):
            break
        vector += diff 
    return vector
    

# model de regression logistique avec descente de gradient
class RegressionLogistique(object):
    def __init__(self):
        self.beta = []
    
    # algorithme de descente de gradient (entrainement)
    def entrainement(self, X, y, learning_rate=1e-4,n_iter=50,tolerance=1e-6, evaluation = "logistic"):
        start = np.random.random(X.shape[1])
        
        if evaluation == "logistic" :
            self.beta = gradient_descent(
                grad_log_vraissemblance, X, y, start, 
                learning_rate=learning_rate, 
                n_iter=n_iter, 
                tolerance=tolerance,
                minProblem=False
            )
        elif evaluation == "rss" :
            self.beta = gradient_descent(
                grad_rss, X, y, start, 
                learning_rate=learning_rate, 
                n_iter=n_iter, 
                tolerance=tolerance
            )
        
    # algorithme d'inference suivat la vraissemblance
    def inference(self, X):
        if self.beta.shape[0] != 0 :
            return logistic(np.dot(X, self.beta))
        return None
    
    

X = np.random.randint(0,2,(30,10))
y = np.random.randint(0,2,30)
model = RegressionLogistique()

model.entrainement(X, y, n_iter=6000)
pred = model.inference(X)

print(y)
print(pred)

[0 0 1 1 0 1 1 1 0 0 1 1 1 0 1 0 0 0 0 1 1 1 0 0 1 1 1 0 0 1]
[0.56203259 0.56686995 0.72115136 0.52730084 0.41684201 0.6547653
 0.56686995 0.6050107  0.56681814 0.64088779 0.3986241  0.69032991
 0.67612493 0.54980241 0.76170858 0.43099152 0.61792983 0.53081433
 0.35197877 0.71631548 0.58041589 0.67417003 0.34654191 0.51283921
 0.82449909 0.74377929 0.40891546 0.4877478  0.38901987 0.52909983]


In [2]:
model.entrainement(X, y, n_iter=60000, evaluation="rss")
pred = model.inference(X)
print(y)
print(pred)

[0 0 1 1 0 1 1 1 0 0 1 1 1 0 1 0 0 0 0 1 1 1 0 0 1 1 1 0 0 1]
[0.40160338 0.42375329 0.84612616 0.39341412 0.29357628 0.87023602
 0.42375329 0.48175067 0.85679297 0.37615604 0.32807972 0.88077447
 0.64998384 0.304859   0.7580709  0.14494334 0.77320572 0.46254288
 0.12445287 0.9148808  0.62205478 0.75780857 0.21742834 0.61390663
 0.97430524 0.94762562 0.32355633 0.72391088 0.10589033 0.66755299]


### ` REGRESSION LINEAIRE `

In [3]:
import numpy as np

# fonction de regression lineaire 
def f(X, b):
    return np.dot(X, b)
    
# fonction d'evaluation du taux d'erreur avec la fonction logistique
def rss(X, y, b):
    a = y - f(X, b)
    return np.dot(a.T, a)
    
# gradient de rss
def grad_rss(X, y, b):
    a = y - f(X, b)
    return  - 2 * np.dot(X.T, a)

# model de regression lineare avec descente de gradient
class RegressionLineaire(object):
    def __init__(self):
        self.beta = []
    
    # algorithme de descente de gradient (entrainement)
    def entrainement(self, X, y, learning_rate=1e-4,n_iter=50,tolerance=1e-6):
        temp = np.ones(y.shape[0])
        temp = temp.reshape((temp.shape[0], 1))
        temp = np.concatenate((temp, X), axis=1)
        
        start = np.random.random(temp.shape[1])
        self.beta = gradient_descent(
                grad_rss, temp, y, start, 
                learning_rate=learning_rate, 
                n_iter=n_iter, 
                tolerance=tolerance
            )
        
    # algorithme d'inference suivat la vraissemblance
    def inference(self, X):
        if self.beta.shape[0] != 0 :
            temp = np.ones(y.shape[0])
            temp = temp.reshape((temp.shape[0], 1))
            temp = np.concatenate((temp, X), axis=1)
            return f(temp, self.beta)
        return None
    
    
model = RegressionLineaire()
model.entrainement(X, y, n_iter=6000)
pred = model.inference(X)

print(y)
print(pred)

[0 0 1 1 0 1 1 1 0 0 1 1 1 0 1 0 0 0 0 1 1 1 0 0 1 1 1 0 0 1]
[0.21070543 0.47262346 0.83461845 0.34157956 0.37547042 0.79322414
 0.47262346 0.50622784 0.24330761 0.51401777 0.35655976 1.14467433
 0.64303    0.23187307 0.93701323 0.19102174 0.34570238 0.35422922
 0.28679184 0.81301357 0.78734515 0.74324565 0.26518696 0.52602824
 1.27026357 1.15324754 0.01020528 0.33350018 0.21147158 0.55145449]
